<a href="https://colab.research.google.com/github/saurabh241930/MachineLearningPractice/blob/master/Keras_LSTM_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt > shakespeare_input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4466k  100 4466k    0     0  4154k      0  0:00:01  0:00:01 --:--:-- 4154k


In [2]:
DATA_PATH = 'shakespeare_input.txt'

data = open(DATA_PATH, 'r').read()
chars = list(set(data)) #set: gets unique values
VOCAB_SIZE = len(chars)

print('chars:\n{}\n\nVOCAB_SIZE: {}'.format(chars, VOCAB_SIZE))

chars:
['A', 'b', 's', 'E', 'T', 'F', 'y', 'U', 'K', '$', ';', '.', 'q', 'C', 'S', 'W', '&', 'l', 'f', 'j', 'n', ',', 'm', '-', 'H', 'k', '\n', 'P', '[', ' ', ':', 'i', ']', "'", 'R', 'N', 'D', 'w', 'o', 'u', 'g', 'p', 'I', 'Y', 'V', '3', 'B', 'O', 'Z', 'h', '!', 'G', 'd', 'Q', 'J', 'X', 'a', '?', 'M', 't', 'c', 'z', 'r', 'v', 'L', 'x', 'e']

VOCAB_SIZE: 67


In [0]:
idx_to_char = {i: char for i, char in enumerate(chars)}
char_to_idx = {char: i for i, char in enumerate(chars)}

In [0]:
import numpy as np

SEQ_LENGTH = 60 #input sequence length
N_FEATURES = VOCAB_SIZE #one hot encoding here, that's why, but deduplicated for clarity

N_SEQ = int(np.floor((len(data) - 1) / SEQ_LENGTH))

X = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))
y = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))

for i in range(N_SEQ):
  X_sequence = data[i * SEQ_LENGTH: (i + 1) * SEQ_LENGTH] #retrieving line 
  X_sequence_ix = [char_to_idx[c] for c in X_sequence] # converting char into index
  input_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
  for j in range(SEQ_LENGTH):
    input_sequence[j][X_sequence_ix[j]] = 1. #one-hot encoding of the input characters
  X[i] = input_sequence
  
  y_sequence = data[i * SEQ_LENGTH + 1: (i + 1) * SEQ_LENGTH + 1] #shifted by 1 to the right
  y_sequence_ix = [char_to_idx[c] for c in y_sequence]
  target_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
  for j in range(SEQ_LENGTH):
    target_sequence[j][y_sequence_ix[j]] = 1. #one-hot encoding of the target characters
  y[i] = target_sequence
  
  


In [8]:
X_sequence

' shock them. Nought shall make us rue,\nIf England to itself '

In [0]:

from keras.models import Sequential
from keras.layers import CuDNNLSTM, TimeDistributed, Dense, Activation

# constant parameter for the model
HIDDEN_DIM = 700 #size of each hidden layer, "each layer has 700 hidden states"
LAYER_NUM = 2 #number of hidden layers, how much were used?
NB_EPOCHS = 200 #max number of epochs to train, "200 epochs"
BATCH_SIZE = 128 
VALIDATION_SPLIT = 0.1 #proportion of the batch used for validation at each epoch


model = Sequential()
model.add(CuDNNLSTM(HIDDEN_DIM, 
               input_shape=(None, VOCAB_SIZE), 
               return_sequences=True))
for _ in range(LAYER_NUM - 1):
  model.add(CuDNNLSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def generate_text(model, length):
  ix = [np.random.randint(VOCAB_SIZE)]
  y_char = [idx_to_char[ix[-1]]]
  X = np.zeros((1, length, VOCAB_SIZE))
  for i in range(length):
    X[0, i, :][ix[-1]] = 1.
    ix = np.argmax(model.predict(X[:, :i+1,:])[0], 1)
    y_char.append(idx_to_char[ix[-1]])
  return ''.join(y_char)

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
# callback to save the model if better
filepath="tgt_model.hdf5"
save_model_cb = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callback to stop the training if no improvement
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=0)
# callback to generate text at epoch end
class generateText(Callback):
    def on_epoch_end(self, batch, logs={}):
        print(generate_text(self.model, 100))
        
generate_text_cb = generateText()

callbacks_list = [save_model_cb, early_stopping_cb, generate_text_cb]

In [0]:
model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=NB_EPOCHS, callbacks=callbacks_list, validation_split=VALIDATION_SPLIT)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 68599 samples, validate on 7623 samples
Epoch 1/200
68599/68599 [==============================] - 119s 2ms/step - loss: 2.3805 - acc: 0.3334 - val_loss: 1.8252 - val_acc: 0.4591

Epoch 00001: val_acc improved from -inf to 0.45908, saving model to tgt_model.hdf5
le shall be the such of the streather the streather the streather the streather the streather the str
Epoch 2/200
68599/68599 [==============================] - 114s 2ms/step - loss: 1.5987 - acc: 0.5188 - val_loss: 1.5541 - val_acc: 0.5327

Epoch 00002: val_acc improved from 0.45908 to 0.53269, saving model to tgt_model.hdf5
 the strength of the reason of the reason
That I have seen the state of the rest of the reason
That I
Epoch 3/200
68599/68599 [==============================] - 115s 2ms/step - loss: 1.4167 - acc: 0.5655 - val_loss: 1.4731 - val_acc: 0.5541

Epoch 00003: val_acc improved fr